## Sales Scraper for Loopnet.com - property-records

 A scraper dedicated to extracting information from the properties records from loopnet.

## ⚠️WARNING⚠️

Overuse of this script may cause the provider to refuse the connection, either temporarily or permanently. To ensure safe usage:
- Keep the `randSleep` function (present in the first cell) unchanged or increase its values for longer pauses.
- Only use when necessary
- This scrapper is kinda slow, and it's intented to be slow, as to avoid ban from the provider


## SETUP

1. install the dependecies
2. put the chromedriver on the drivers folder
3. all done


## Usage

Description of what each cell does is privided direct above, and within the cell as a comment on the first lines.



This cell is the setup step. It loads all the necessary packages and opens the browser for web scraping. It navigates to the URL specified in ```driver.get(<URL>)```, which should be the first page of the property listings.

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import numpy as np
import random
import json

def randSleep():
    time.sleep(random.uniform(1, 2))


def pretty_print(d, indent=0):
    for key, value in d.items():
        print("  " * indent + str(key) + ":")
        if isinstance(value, dict):
            pretty_print(value, indent + 1)
        else:
            print("  " * (indent + 1) + str(value))




# service = Service("./drivers/chromedriver")
# driver = webdriver.Chrome()
# driver.page_load_strategy = 'normal'  # Default behavior, waits for complete page load
# wait = WebDriverWait(driver, 10) # mistery wait

driver = webdriver.Chrome(executable_path=r"/Users/kylewang/Desktop/MLSA_2024/Upwork/chromedriver")
driver.page_load_strategy = 'normal'  # Default behavior, waits for complete page load

driver.get("https://www.loopnet.com/sitemap/property-records/texas/abbott/")

/var/folders/2p/mkvb6kbd08l30n0fx72jcwz80000gn/T/ipykernel_39088/2551452670.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r"/Users/kylewang/Desktop/MLSA_2024/Upwork/chromedriver")


With the browser open on the first page of the property listings, run the cell below to scrape and save all the links for the individual listings.

In [3]:
# Link scrapping process

def get_links():
    box_info = driver.find_elements(By.CSS_SELECTOR, ".property-columns")
    links = box_info[0].find_elements(By.TAG_NAME, "a")
    links = [link.get_attribute("href") for link in links]
    #for i in range(0, len(links)):
    #    print(links[i])
    
    return links

#get_links()

def go_next():
    next_menu = driver.find_element(By.CSS_SELECTOR, ".paging")
    next_button = next_menu.find_elements(By.CSS_SELECTOR, ".beforeellipsisli.caret-right-large")
    if len(next_button) > 0:
        next_button = next_button[0].click()
        return True
    else:
        print("No more pages")
        return False

def randSleep():
    time.sleep(random.uniform(1, 2))


links = np.array([])
while True:
    randSleep()
    links = np.append(links, get_links())
    print(links.shape)
    cond = go_next()
    if cond == False:
        break

links = np.array(links).flatten()

print(len(links))

link_file = open("links.txt", "w")
for i in range(0, len(links)):
    print(links[i])
    link_file.write(links[i] + "\n")




(120,)
(240,)
(360,)
(477,)
No more pages
477
https://www.loopnet.com/property/219-s-1st-st-abbott-tx-76621/48217-109541/
https://www.loopnet.com/property/1094-ih-35-s-abbott-tx-76621/48217-108667/
https://www.loopnet.com/property/1197-ih-35-s-abbott-tx-76621/48217-108675/
https://www.loopnet.com/property/i-35-s-i-35-s-s-abbott-tx-76621/48217-105250/
https://www.loopnet.com/property/1221-n-pecan-st-abbott-tx-76621/48217-108794/
https://www.loopnet.com/property/-hc-3140-hc-abbott-tx-76621/48217-108701/
https://www.loopnet.com/property/hc-2340-abbott-tx-76621/48217-125324/
https://www.loopnet.com/property/hc-23412342-abbott-tx-76621/48217-108933/
https://www.loopnet.com/property/i-35-s-hc-2455-abbott-tx-76621/48217-108826/
https://www.loopnet.com/property/hc-2341-abbott-tx-76621/48217-109065/
https://www.loopnet.com/property/417-hcr-2302-n-abbott-tx-76621/48217-108459/
https://www.loopnet.com/property/hc-2455-abbott-tx-76621/48217-108852/
https://www.loopnet.com/property/i-35-e-serv-rd-a

The cell below is to load the links from a file.

In [4]:
# link loading process
link_list = np.loadtxt("links.txt", dtype=str)

print("Total links: ", len(link_list))

Total links:  442


This cell initializes the dictionary that will contain all the information scraped from the individual listings, as well as the counter for the page iterations. These variables are initialized separately so that the scraping process can be performed in parts if necessary.

In [5]:
# variables initializing
all_data = {}
page_index = 0

This cell scrapes the actual information from each listing page using the loaded links.

In [ ]:
# Proper data scrapping
init_page = 0
final_page = len(link_list)

def get_infos():
    # this function get the facts from the property
    infobox = driver.find_element(By.ID, "detailsInf")
    info_names = infobox.find_elements(By.CSS_SELECTOR, ".assessment-key")
    info_values = infobox.find_elements(By.CSS_SELECTOR, ".assessment-value")
    info_names_ret = []
    info_vals_ret = []
    for i in range(0, len(info_names)):
        info_names_ret.append(info_names[i].text)
        info_vals_ret.append(info_values[i].text)
    
    return info_names_ret, info_vals_ret


def get_listing():
    # this part gets the infos from the listing
    infobox = driver.find_elements(By.CSS_SELECTOR, ".highlights.include-in-page.demographics")
    #print(len(infobox))
    break_cond = True
    for i in range(0, len(infobox)):
        if "LISTING" in infobox[i].text:
            infobox = infobox[i]
            break_cond = False

    if break_cond:
        print("No listing info")
        return None, None
        
    infos = infobox.find_elements(By.CSS_SELECTOR, ".flex-half.text-uppercase")
    info_names = infos[0].text.split("\n")
    info_values = infos[1].text.split("\n")
    return info_names, info_values

def get_link():
    return driver.current_url

def get_demo():
    # this part gets the demographics informations if avaliable
    infobox = driver.find_elements(By.CSS_SELECTOR, ".highlights.include-in-page.demographics")

    break_cond = True
    for i in range(0, len(infobox)):
        if "DEMOGRAPHICS" in infobox[i].text:
            infobox = infobox[i]
            break_cond = False
            break
    
    if break_cond:
        print("No demo info")
        return None, None
    
    table = infobox.find_elements(By.CSS_SELECTOR, ".flex")
    distaces = table[0].text.split("\n")
    infos = []
    for i in range(1, len(table)):
        temp = table[i].text.split("\n")
        temp = temp
        infos.append(temp)


    val_names = []
    val_vals = []
    for i in range(0, len(distaces)):
        for j in range(0, len(infos)):
            val_names.append((distaces[i] + " | " + infos[j][0]).lower())
            val_vals.append(infos[j][i+1])

    #for i in range(0, len(val_names)):
    #    print(val_names[i], ":", val_vals[i])

    return val_names, val_vals


def get_data():
    info_names, info_vals = get_infos()
    listing_names, listing_vals = get_listing()
    demo_names, demo_vals = get_demo()

    all_data = {}
    all_data["link"] = get_link()
    for i in range(0, len(info_names)):
        all_data[info_names[i].lower()] = info_vals[i]

    if listing_names != None:
        for i in range(0, len(listing_names)):
            all_data[listing_names[i].lower()] = listing_vals[i]

    if demo_names != None:
        for i in range(0, len(demo_names)):
            all_data[demo_names[i]] = demo_vals[i]

    #pretty_print(all_data)

    return all_data


while page_index < final_page:
    print("Page:", page_index, links[page_index])
    driver.get(link_list[page_index])
    main_text = driver.find_element(By.TAG_NAME, "h1")
    if "Sorry" in main_text.text:
        print("Sorry, this page is not avaliable")
        all_data[page_index] = None
    else:
        randSleep()
        all_data[page_index] = get_data()
        print(all_data[page_index])

    page_index += 1


output = open("records.json", "w")
json.dump(all_data, output, indent=4)
#output.close()

Page: 0 https://www.loopnet.com/property/219-s-1st-st-abbott-tx-76621/48217-109541/
No listing info
No demo info
{'link': 'https://www.loopnet.com/property/219-s-1st-st-abbott-tx-76621/48217-109541/', 'address': '219 S 1St St', 'apn/parcel id': '109541', 'subdivision name': 'J Pool A-', 'land use': 'Taxexempt', 'county': 'Hill', 'zoning': 'X', 'state': 'Texas', 'flood zone': '48217C0425D', 'lot size': '7.51 AC', 'total size': '86,343 SF'}
Page: 1 https://www.loopnet.com/property/1094-ih-35-s-abbott-tx-76621/48217-108667/
{'link': 'https://www.loopnet.com/property/1094-ih-35-s-abbott-tx-76621/48217-108667/', 'address': '1094 Ih 35 S', 'apn/parcel id': '108667', 'subdivision name': 'J Frederick A-', 'land use': 'Commercialnec', 'legal description': 'J FREDERICK A-311 TR 4 22.11 AC', 'zoning': 'X', 'county': 'Hill', 'flood zone': '48217C0425D', 'state': 'Texas', 'tax year': '2023', 'lot size': '22.11 AC', 'assessed year': '2024', 'submarket': 'Central Texas Area', 'census': '960700', 'mar

No listing info
No demo info
{'link': 'https://www.loopnet.com/property/hc-2341-abbott-tx-76621/48217-109065/', 'address': 'Hc 2341', 'apn/parcel id': '109065', 'subdivision name': 'J Obrian A-', 'land use': 'Agriculturalland', 'county': 'Hill', 'zoning': 'A', 'state': 'Texas', 'flood zone': '48217C0425D', 'lot size': '104.32 AC'}
Page: 10 https://www.loopnet.com/property/417-hcr-2302-n-abbott-tx-76621/48217-108459/
No listing info
{'link': 'https://www.loopnet.com/property/417-hcr-2302-n-abbott-tx-76621/48217-108459/', 'address': '417 Hcr 2302 N', 'land use': 'Farms', 'subdivision name': 'W Burnback A-', 'zoning': 'A', 'legal description': 'W BURNBACK A-33 TR 10 86.819 AC', 'flood zone': '48217C0550C', 'county': 'Hill', 'tax year': '2023', 'state': 'Texas', 'assessed year': '2024', 'lot size': '86.82 AC', 'census': '960700', 'apn/parcel id': '108459', 'total size': '3,303 SF', '1 mile | 2024 total population': '28', '1 mile | 2029 population': '30', '1 mile | pop growth 2024-2029': '+

No listing info
No demo info
{'link': 'https://www.loopnet.com/property/2340-fm-1304-abbott-tx-76621/48217-108656/', 'address': '2340 Fm 1304', 'apn/parcel id': '108656', 'subdivision name': 'S Fitzgerald A-', 'land use': 'Agriculturalland', 'county': 'Hill', 'zoning': 'X', 'state': 'Texas', 'flood zone': '48217C0550C', 'lot size': '151.09 AC', 'total size': '14,250 SF'}
Page: 19 https://www.loopnet.com/property/hc-3210-abbott-tx-76621/48217-109239/
No listing info
{'link': 'https://www.loopnet.com/property/hc-3210-abbott-tx-76621/48217-109239/', 'address': 'Hc 3210', 'land use': 'Vacantlandnec', 'subdivision name': 'J Wallace A-', 'zoning': 'X', 'legal description': 'J WALLACE A-938 TR 18 & 18A 189.50 AC', 'flood zone': '48217C0425D', 'county': 'Hill', 'tax year': '2023', 'state': 'Texas', 'assessed year': '2024', 'lot size': '189.50 AC', 'census': '960700', 'apn/parcel id': '109239', '1 mile | 2024 total population': '0', '1 mile | 2029 population': '0', '1 mile | pop growth 2024-202

No listing info
No demo info
{'link': 'https://www.loopnet.com/property/hc-3111-w-abbott-tx-76621/48217-108871/', 'address': 'Hc 3111 W', 'apn/parcel id': '108871', 'subdivision name': 'Leon Co Sch Ld A-517', 'land use': 'Agriculturalland', 'county': 'Hill', 'zoning': 'A', 'state': 'Texas', 'flood zone': '48217C0425D', 'lot size': '164 AC'}
Page: 32 https://www.loopnet.com/property/-fm-1947-hc-abbott-tx-76621/48217-109075/
Sorry, this page is not avaliable
Page: 33 https://www.loopnet.com/property/i-35-se-serv-rd-abbott-tx-76621/48217-108901/
{'link': 'https://www.loopnet.com/property/i-35-se-serv-rd-abbott-tx-76621/48217-108901/', 'address': 'I-35 Se Serv Rd', 'apn/parcel id': '108901', 'subdivision name': 'Leon Co Sch Ld A-517', 'land use': 'Agriculturalland', 'legal description': 'LEON CO SCH LD A-517 #48 TR 2 83.30 AC', 'zoning': 'X', 'county': 'Hill', 'flood zone': '48217C0405D', 'state': 'Texas', 'tax year': '2023', 'lot size': '83.30 AC', 'assessed year': '2024', 'submarket': 'C

Sorry, this page is not avaliable
Page: 42 https://www.loopnet.com/property/hc-2317-abbott-tx-76621/48217-108488/
No listing info
{'link': 'https://www.loopnet.com/property/hc-2317-abbott-tx-76621/48217-108488/', 'address': 'Hc 2317', 'land use': 'Agriculturalland', 'subdivision name': 'W Bass A-', 'zoning': 'X', 'legal description': 'W BASS A-34 TR 5 119.00 AC', 'flood zone': '48217C0550C', 'county': 'Hill', 'tax year': '2023', 'state': 'Texas', 'assessed year': '2024', 'lot size': '119 AC', 'census': '960700', 'apn/parcel id': '108488', '1 mile | 2024 total population': '35', '1 mile | 2029 population': '38', '1 mile | pop growth 2024-2029': '+ 8.57%', '1 mile | average age': '42', '1 mile | 2024 total households': '13', '1 mile | hh growth 2024-2029': '+ 7.69%', '1 mile | median household inc': '$81,250', '1 mile | avg household size': '2.70', '1 mile | 2024 avg hh vehicles': '2.00', '1 mile | median home value': '$166,666', '1 mile | median year built': '1979', '3 mile | 2024 total

...

No listing info
No demo info
{'link': 'https://www.loopnet.com/property/680-hcr-3112-abbott-tx-76621/48217-108720/', 'address': '680 Hcr 3112', 'apn/parcel id': '108720', 'subdivision name': 'S Holloway A-', 'land use': 'Vacantlandnec', 'county': 'Hill', 'zoning': 'X', 'state': 'Texas', 'flood zone': '48217C0575D', 'lot size': '138.47 AC', 'total size': '1,341 SF'}
Page: 64 https://www.loopnet.com/property/hc-3206-abbott-tx-76621/48217-109605/
No listing info
{'link': 'https://www.loopnet.com/property/hc-3206-abbott-tx-76621/48217-109605/', 'address': 'Hc 3206', 'land use': 'Vacantlandnec', 'subdivision name': 'S Holloway A-', 'zoning': 'X', 'legal description': 'S HOLLOWAY A-367 TR 8 & 8B 108.399 AC', 'flood zone': '48217C0575D', 'county': 'Hill', 'tax year': '2023', 'state': 'Texas', 'assessed year': '2024', 'lot size': '108.40 AC', 'census': '960700', 'apn/parcel id': '109605', '1 mile | 2024 total population': '14', '1 mile | 2029 population': '15', '1 mile | pop growth 2024-2029':

No listing info
{'link': 'https://www.loopnet.com/property/hc-2320-abbott-tx-76621/48217-108835/', 'address': 'Hc 2320', 'apn/parcel id': '108835', 'subdivision name': 'Leon Co Sch Ld A-517', 'land use': 'Agriculturalland', 'legal description': 'LEON CO SCH LD A-517 #25 TR 1 132.573 AC', 'zoning': 'X', 'county': 'Hill', 'flood zone': '48217C0400D', 'state': 'Texas', 'tax year': '2023', 'lot size': '132.57 AC', 'assessed year': '2024', 'submarket': 'Central Texas Area', 'census': '960700', 'market': 'Other Market Areas', '1 mile | 2024 total population': '0', '1 mile | 2029 population': '0', '1 mile | pop growth 2024-2029': '0.00%', '1 mile | average age': '0', '1 mile | 2024 total households': '0', '1 mile | hh growth 2024-2029': '0.00%', '1 mile | median household inc': '$0', '1 mile | avg household size': '.00', '1 mile | 2024 avg hh vehicles': '.00', '1 mile | median home value': '$0', '1 mile | median year built': '0', '3 mile | 2024 total population': '180', '3 mile | 2029 populat

No listing info
No demo info
{'link': 'https://www.loopnet.com/property/2302-s-hcr-abbott-tx-76621/48217-108940/', 'address': '2302 S Hcr', 'apn/parcel id': '108940', 'subdivision name': 'D Laird A-', 'land use': 'Agriculturalland', 'county': 'Hill', 'zoning': 'A', 'state': 'Texas', 'flood zone': '48217C0575D', 'lot size': '100 AC'}
Page: 83 https://www.loopnet.com/property/hc-3118-abbott-tx-76621/48217-109279/
No listing info
No demo info
{'link': 'https://www.loopnet.com/property/hc-3118-abbott-tx-76621/48217-109279/', 'address': 'Hc 3118', 'apn/parcel id': '109279', 'subdivision name': 'S White A-', 'land use': 'Agriculturalland', 'county': 'Hill', 'zoning': 'X', 'state': 'Texas', 'flood zone': '48217C0425D', 'lot size': '100 AC'}
Page: 84 https://www.loopnet.com/property/hc-3110-s-abbott-tx-76621/48217-109250/
No listing info
{'link': 'https://www.loopnet.com/property/hc-3110-s-abbott-tx-76621/48217-109250/', 'address': 'Hc 3110 S', 'land use': 'Vacantlandnec', 'subdivision name': 

No listing info
No demo info
{'link': 'https://www.loopnet.com/property/hc-3206-abbott-tx-76621/48217-109598/', 'address': 'Hc 3206', 'apn/parcel id': '109598', 'subdivision name': 'C Gilbreath A-', 'land use': 'Vacantlandnec', 'county': 'Hill', 'zoning': 'A', 'state': 'Texas', 'flood zone': '48217C0575D', 'lot size': '97.06 AC'}
Page: 92 https://www.loopnet.com/property/hc-2320-abbott-tx-76621/48217-108694/
No listing info
No demo info
{'link': 'https://www.loopnet.com/property/hc-2320-abbott-tx-76621/48217-108694/', 'address': 'Hc 2320', 'apn/parcel id': '108694', 'subdivision name': 'P George A-', 'land use': 'Vacantlandnec', 'county': 'Hill', 'zoning': 'A', 'state': 'Texas', 'flood zone': '48217C0400D', 'lot size': '97 AC', 'total size': '1,088 SF'}
Page: 93 https://www.loopnet.com/property/at-hc-2302-s-abbott-tx-76621/48217-115912/
No listing info
{'link': 'https://www.loopnet.com/property/at-hc-2302-s-abbott-tx-76621/48217-115912/', 'address': 'At Hc 2302 S', 'land use': 'Agricul

No listing info
No demo info
{'link': 'https://www.loopnet.com/property/hc-3106-abbott-tx-76621/48217-105668/', 'address': 'Hc 3106', 'apn/parcel id': '105668', 'subdivision name': 'M Barker A-', 'land use': 'Agriculturalland', 'county': 'Hill', 'zoning': 'X', 'state': 'Texas', 'flood zone': '48217C0425D', 'lot size': '118.50 AC'}
Page: 108 https://www.loopnet.com/property/hc-2302-n-abbott-tx-76621/48217-108465/
No listing info
No demo info
{'link': 'https://www.loopnet.com/property/hc-2302-n-abbott-tx-76621/48217-108465/', 'address': 'Hc 2302 N', 'apn/parcel id': '108465', 'subdivision name': 'W Burnback A-', 'land use': 'Agriculturalland', 'county': 'Hill', 'zoning': 'X', 'state': 'Texas', 'flood zone': '48217C0400D', 'lot size': '118.10 AC'}
Page: 109 https://www.loopnet.com/property/624-hcr-2302-s-abbott-tx-76621/48217-125322/
No listing info
No demo info
{'link': 'https://www.loopnet.com/property/624-hcr-2302-s-abbott-tx-76621/48217-125322/', 'address': '624 Hcr 2302 S', 'apn/parc

This cell loads the JSON file containing the scraped information.

In [ ]:
# Table generator
import pandas as pd

all_data = json.load(open("records.json"))


The cell below saves the unfiltered data in a table format.

In [ ]:
#saving the unfildered data

# Flatten the JSON data
flattened_data = pd.json_normalize(all_data.values())

# remove lines where pages were missing
flattened_data = pd.DataFrame(flattened_data)
flattened_data = flattened_data.dropna(axis=0, how="all")


# Convert to DataFrame
df = pd.DataFrame(flattened_data)

# Save to Excel
df.to_excel("unfiltered_records.xlsx", index=False)


# this prints the avaialble fields for the filtering

print("Columns:")
print(df.columns)

This final cell saves the filtered data in a table format, based on the fields specified in the `my_fields` list.

In [ ]:
# # saving filtered data

# my_fields = ["link","address", "subdivision name","legal description","county", "state", "lot #" , "submarket", "land use", 'zoning', 'flood zone', 'tax year', 'assessed year', "census", "total size", 'listing date', 'sale/lease', '1 mile | 2029 population', '1 mile | pop growth 2024-2029', '1 mile | average age', '1 mile | 2024 total households', '1 mile | hh growth 2024-2029', '1 mile | median household inc', '1 mile | avg household size', '1 mile | 2024 avg hh vehicles', '1 mile | median home value', '1 mile | median year built', '3 mile | 2024 total population', '3 mile | 2029 population', '3 mile | pop growth 2024-2029', '3 mile | average age', '3 mile | 2024 total households', '3 mile | hh growth 2024-2029', '3 mile | median household inc', '3 mile | avg household size', '3 mile | 2024 avg hh vehicles', '3 mile | median home value', '3 mile | median year built', '5 mile | 2024 total population', '5 mile | 2029 population', '5 mile | pop growth 2024-2029', '5 mile | average age', '5 mile | 2024 total households', '5 mile | hh growth 2024-2029', '5 mile | median household inc', '5 mile | avg household size', '5 mile | 2024 avg hh vehicles', '5 mile | median home value', '5 mile | median year built']

# df_to_save = df[my_fields]
# df_to_save.to_excel("filtered_records.xlsx", index=False)
